In [1]:
import torch
a = torch.tensor(1.0, requires_grad=True)
b = a.clone()
# a和b不是同一个对象
print(id(a), id(b))
# 也不指向同一块内存地址
print(a.data_ptr(), b.data_ptr())
# 但b的requires_grad属性和a的一样，同样是True
print(a.requires_grad, b.requires_grad)

c = a * 2
c.backward()
print(a.grad)

b = b + 2
d = b * 3
d.backward()
# b是普通节点而非叶子结点梯度值不会被保存, 但是b在计算图中，其梯度会传递给a
print(b.grad, a.grad)


5183378112 5162402656
4358384768 4358787008
True True
tensor(2.)
None tensor(5.)


/var/folders/jb/g3ylc11574sd3g6w3t57g5rm0000gn/T/ipykernel_22502/3133686898.py:19: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at /Users/runner/work/_temp/anaconda/conda-bld/pytorch_1662188943573/work/build/aten/src/ATen/core/TensorBody.h:483.)
  print(b.grad, a.grad)


In [6]:
import torch

a = torch.tensor(1.0, requires_grad=True)
b = a
print(id(a), id(b))
print(a.data_ptr(), b.data_ptr())
print(a.requires_grad, b.requires_grad)

c = a * 2
c.backward()
print(a.grad)

# b = b + 2
d = b * 3
d.backward()
print(b.grad, a.grad)

4401773920 4401773920
4952537088 4952537088
True True
tensor(2.)
tensor(5.) tensor(5.)


In [1]:
import torch
from torch import nn

class SelfDefinedSigmoid(torch.autograd.Function):

    @staticmethod
    def forward(ctx, inp):
        result = torch.divide(torch.tensor(1), (1 + torch.exp(-inp)))
        ctx.save_for_backward(result)
        return result

    @staticmethod
    def backward(ctx, grad_output):
        # ctx.saved_tensors is tuple (tensors, grad_fn)
        result, = ctx.saved_tensors
        return grad_output * result * (1 - result)


class Sigmoid(nn.Module):

    def __init__(self):
        super().__init__()

    def forward(self, x):
        out = SelfDefinedSigmoid.apply(x)
        return out


torch.manual_seed(0)
sg = Sigmoid()
inputs = torch.nn.Parameter(torch.tensor(2.))
outputs = sg(inputs)
outputs.backward(torch.ones_like(inputs), retain_graph=True)

print(outputs, inputs, inputs.grad)

tensor(0.8808, grad_fn=<SelfDefinedSigmoidBackward>) Parameter containing:
tensor(2., requires_grad=True) tensor(0.1050)


In [1]:
# 冒泡排序
rand_list = [0, 1, 4, 3, 2, 5, 6, 7, 9, 8]
n = len(rand_list)
for i in range(1, n):
    for j in range(0, n - i):
        if rand_list[j] > rand_list[j + 1]:
            rand_list[j], rand_list[j + 1] = rand_list[j + 1], rand_list[j]
print(rand_list)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


In [ ]:
import numpy as np
import matplotlib.pyplot as plot

fig = plot.figure(1, figsize=(2, 2), dpi=300)
image = np.array([[1, 1, 1, 1], [2, 2, 2, 2], [3, 3, 3, 3], [4, 4, 4, 4]])
plot.imshow(image, cmap='gray')
plot.margins(0., 0.)  # 让轴中的内容紧贴轴
plot.axis('off')
plot.subplots_adjust(0., 0., 1., 1., 0., 0.)  # 让全部轴所处的区域与图完全重合
fig.savefig('image.tiff', bbox_inches='tight', pad_inches=0., transparent=True)
plot.show()
plot.close(fig=fig)